In [3]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=5)


KMeans(n_clusters=3)

k-means + LSTM 完成学习路径推荐

In [17]:
import pandas as pd

java = pd.read_csv('java_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
java

,user_id,item_id,behavior,timestamp
0,1,305,watch,1406863244
1,1,172,done,1410874261
2,1,54,done,1410874265
3,1,54,duration,1410874275
4,1,314,done,1410913417
...,...,...,...,...
2489,183,294,done,1429083169
2490,183,294,tag,1429083255
2491,183,294,duration,1429083393
2492,183,357,done,1429084813


In [8]:
java.groupby(['user_id','behavior']).count().reset_index()

,user_id,behavior,item_id,timestamp
0,1,done,31,31
1,1,duration,27,27
2,1,tag,6,6
3,1,watch,12,12
4,2,done,1,1
...,...,...,...,...
747,183,done,4,4
748,183,duration,2,2
749,183,enter,1,1
750,183,tag,2,2


In [11]:
beh_count = java.groupby(['user_id','behavior']).count().reset_index()

In [9]:
java['behavior'].nunique()

5

In [21]:
java_user_count = java.groupby('user_id').count().reset_index()[['user_id','item_id']]
java_inter = java[java['user_id'].isin(java_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()
java_inter

,user_id,item_id
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310..."
1,2,"[363, 1]"
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29..."
3,4,"[363, 1]"
4,5,"[1, 294, 363]"
...,...,...
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]"
179,180,[1]
180,181,"[363, 1]"
181,182,"[294, 54, 1]"


In [26]:
java_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
java_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
java_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
java_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
java_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x : len(x))
java_inter = java_inter[java_inter['inter_len']>=3]
java_inter = java_inter.fillna(0)
java_inter

,user_id,item_id,done_count,watch_count,enter_count,tag_count,dur_count,inter_len
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",31.0,0.0,0.0,0.0,0.0,33
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",0.0,0.0,0.0,6.0,0.0,11
4,5,"[1, 294, 363]",1.0,0.0,0.0,0.0,0.0,3
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",0.0,0.0,1.0,0.0,0.0,33
6,7,"[294, 1, 363]",0.0,0.0,0.0,1.0,0.0,3
...,...,...,...,...,...,...,...,...
176,177,"[1, 337, 294]",10.0,0.0,0.0,0.0,0.0,3
177,178,"[298, 311, 1, 331]",0.0,0.0,0.0,0.0,0.0,4
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",0.0,0.0,0.0,11.0,0.0,8
181,182,"[294, 54, 1]",0.0,0.0,0.0,0.0,0.0,3


In [38]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=5)
train_x = java_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['cat'] = predict_y
java_inter

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,item_id,done_count,watch_count,enter_count,tag_count,dur_count,inter_len,cat
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",31.0,0.0,0.0,0.0,0.0,33,3
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",0.0,0.0,0.0,6.0,0.0,11,4
4,5,"[1, 294, 363]",1.0,0.0,0.0,0.0,0.0,3,2
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",0.0,0.0,1.0,0.0,0.0,33,1
6,7,"[294, 1, 363]",0.0,0.0,0.0,1.0,0.0,3,2
...,...,...,...,...,...,...,...,...,...
176,177,"[1, 337, 294]",10.0,0.0,0.0,0.0,0.0,3,2
177,178,"[298, 311, 1, 331]",0.0,0.0,0.0,0.0,0.0,4,2
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",0.0,0.0,0.0,11.0,0.0,8,4
181,182,"[294, 54, 1]",0.0,0.0,0.0,0.0,0.0,3,2


In [62]:
java_inter[java_inter['user_id']==62]

,user_id,item_id,done_count,watch_count,enter_count,tag_count,dur_count,inter_len,cat
61,62,"[294, 1, 295, 35, 297, 300, 301, 306, 307, 311...",0.0,0.0,1.0,0.0,0.0,45,1


In [64]:
alldata = []
for user in java_inter['user_id'].tolist():
    print(user)
    seq_len = len(java_inter[java_inter['user_id']==user]['item_id'].item())
    total_seq = java_inter[java_inter['user_id']==1]['item_id'].item().tolist()
    cat = java_inter[java_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(inter_seq)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['inter_seq','cat','target','pred_path'])
java_new

1
3
5
6
7
8
9
15
16
21
24
25
26
28
29
35
45
46
47
48
49
51
52
53
54
55
56
61
62
63
65
66
67
68
69
70
71
72
75
76
77
78
79
82
83
85
87
88
90
92
94
95
98
99
104
106
107
109
110
111
112
113
115
116
117
126
127
129
130
132
133
134
136
137
138
139
141
142
149
150
153
157
158
159
160
161
162
163
165
169
176
177
178
179
182
183


,inter_seq,cat,target,pred_path
0,[305],3,172,[172]
1,[305],3,54,"[172, 54]"
2,[305],3,314,"[172, 54, 314]"
3,[305],3,1,"[172, 54, 314, 1]"
4,[305],3,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...
9854,[305],1,54,"[172, 54]"
9855,[305],1,314,"[172, 54, 314]"
9856,"[305, 172]",1,54,[54]
9857,"[305, 172]",1,314,"[54, 314]"


In [70]:
java_test = java_new[java_new['pred_path'].apply(lambda x: len(x))<=5].sample(frac=0.25)
java_train = java_new[~java_new.index.isin(java_test)]
java_test

,inter_seq,cat,target,pred_path
527,"[305, 172, 54, 314, 1]",4,352,"[350, 352]"
1547,"[305, 172, 54, 314, 1]",2,302,"[350, 352, 301, 302]"
8229,[305],2,314,"[172, 54, 314]"
3571,"[305, 172, 54, 314, 1, 350, 352, 301, 302]",2,310,[310]
63,"[305, 172, 54]",3,350,"[314, 1, 350]"
...,...,...,...,...
8868,"[305, 172, 54, 314]",0,301,"[1, 350, 352, 301]"
6921,"[305, 172, 54, 314, 1, 350]",2,302,"[352, 301, 302]"
973,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",1,295,"[294, 295]"
8123,"[305, 172, 54]",3,350,"[314, 1, 350]"


In [71]:
java_test.to_csv('java_test.csv',header=True,sep='\t',index=False)
java_train.to_csv('java_train.csv',header=True,sep='\t',index=False)